In [1]:
import sys

In [2]:
TXT_PROJ_ROOT = "/home/gml/Documents/Master/M1_BIBS/S2/Structurale/"

In [3]:
sys.path.append(TXT_PROJ_ROOT)

In [8]:
from piestruct.utils.customobjs import (ObjDict as odict, Path as path)

In [9]:
import freesasa

In [10]:
proj_root = path(TXT_PROJ_ROOT)

In [11]:
pdb_files = proj_root.joinpath("data").dglob("*.pdb")
pdb_files

{'1he8.pdb': PosixPath('/home/gml/Documents/Master/M1_BIBS/S2/Structurale/data/1he8.pdb'),
 '1HE8_AB_c.pdb': PosixPath('/home/gml/Documents/Master/M1_BIBS/S2/Structurale/data/1HE8_AB_c.pdb'),
 '1e8y.pdb': PosixPath('/home/gml/Documents/Master/M1_BIBS/S2/Structurale/data/1e8y.pdb'),
 '121p.pdb': PosixPath('/home/gml/Documents/Master/M1_BIBS/S2/Structurale/data/121p.pdb')}

## Créer les structures "freesasa"

In [12]:
structures = odict({ 
    name.replace(".pdb", ""): freesasa.Structure(location.abs) 
    for name, location in pdb_files.items() 
})
structures

{'1he8': <freesasa.Structure at 0x7fe4644faf00>,
 '1HE8_AB_c': <freesasa.Structure at 0x7fe4645010c0>,
 '1e8y': <freesasa.Structure at 0x7fe464501de0>,
 '121p': <freesasa.Structure at 0x7fe464501ed0>}

## Faire le calcul 
Je ne sais pas de quoi 

In [13]:
sasas = odict({
    name: freesasa.calc(struct)
    for name, struct in structures.items()
})
sasas

{'1he8': <freesasa.Result at 0x7fe464518d90>,
 '1HE8_AB_c': <freesasa.Result at 0x7fe464518910>,
 '1e8y': <freesasa.Result at 0x7fe464518930>,
 '121p': <freesasa.Result at 0x7fe464518e90>}

## Calculer les aires

In [14]:
resid_areas = odict({
    name: sasa.residueAreas()
    for name, sasa in sasas.items()
})

In [15]:
monomere1 = resid_areas["121p"]
monomere2 = resid_areas["1e8y"]

In [16]:
monomere1.keys(), monomere2.keys()

(dict_keys(['A']), dict_keys(['A']))

In [17]:
for value in monomere1["A"].values():
    if value.relativeTotal > 1.0:
        print("Marco non ci dice la verità")

In [18]:
sum(residu.relativeTotal for residu in monomere1["A"].values())

53.29124167145358

In [19]:
sum(sasa.total for sasa in monomere1["A"].values())

8720.956397446842

In [20]:
complexe = sasas['1HE8_AB_c'].residueAreas()

In [21]:
complexe.keys()

dict_keys(['A', 'B'])

In [77]:
dir(monomere1["A"]["1"])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'apolar',
 'hasRelativeAreas',
 'mainChain',
 'polar',
 'relativeApolar',
 'relativeMainChain',
 'relativePolar',
 'relativeSideChain',
 'relativeTotal',
 'residueNumber',
 'residueType',
 'sideChain',
 'total']